# Loading the reference images

In [ ]:
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from cloth_tools.dataset.format import load_competition_observation

data_dir = Path("data")
dataset_dir = data_dir / "cloth_competition_dataset_0001_segmented"
references_dir = data_dir / "cloth_competition_references_0001"
info_csv = data_dir / "cloth_competition_dataset_0001_info.csv"


In [ ]:
os.path.exists(references_dir)

In [ ]:
reference_sample_dirs = [references_dir / dirname for dirname in sorted(os.listdir(references_dir))]
reference_sample_dirs

In [ ]:
reference_observation_dirs = [sample_dir / "observation_result" for sample_dir in reference_sample_dirs]

# observation_dirs = [dataset_dir / ref_dir for ref_dir in sorted(os.listdir(dataset_dir))]
len(reference_observation_dirs)

In [ ]:
import json

reference_images = {}
reference_surface_areas = {}

for cloth_id, observation_dir in enumerate(reference_observation_dirs):
    observation = load_competition_observation(observation_dir)

    results_file = observation_dir / "result.json"

    if results_file.exists():
        with open(results_file, "r") as f:
            result = json.load(f)
        surface_area = result["coverage"]
        reference_images[f"Reference {cloth_id} ({surface_area:.3f} m2)"] = observation.image_left
        reference_surface_areas[cloth_id] = surface_area
    else:
        reference_images[f"Reference {cloth_id}"] = observation.image_left


In [ ]:
import matplotlib.pyplot as plt

def display_images(images_dict):
    """Displays reference images in a grid layout.

    Args:
        reference_images (dict): A dictionary where keys are cloth IDs and
                                 values are the corresponding images.
    """

    n_images = len(images_dict)
    cols = 5  # Adjust the number of columns as desired
    rows = (n_images // cols) + (n_images % cols > 0)  # Calculate rows for the grid

    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols * 4, rows * 3))  # Create subplots

    cloth_ids = list(images_dict.keys())
    images = list(images_dict.values())

    for i, ax in enumerate(axes.flat):
        if i < n_images:
            image = images[i]
            cloth_id = cloth_ids[i]
            ax.imshow(image)
            ax.set_title(f"{cloth_id}")
            ax.axis("off")  # Hide unused subplots
        # else:

    plt.tight_layout()
    plt.show()

# Example usage with your dictionary:
display_images(reference_images) 

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df_info = pd.read_csv(info_csv)

df_info.head()

In [ ]:
# TODO iterate over all sample dirs in cloth_competition_dataset_0001 and create dataframe with coverage etc, then merge with df

sample_dirs = [dataset_dir / dirname for dirname in sorted(os.listdir(dataset_dir))]

print(len(sample_dirs))

In [ ]:
# result_observations = []
result_images_dict = {}
surface_areas = []

for sample_dir in sample_dirs:
    observation_results_dir = sample_dir / "observation_result"
    observation = load_competition_observation(observation_results_dir)
    result_images_dict[f"{sample_dir.name}"] = observation
    # result_observations.append(observation)
    results_file = observation_results_dir / "result.json"

    with open(results_file, "r") as f:
            result = json.load(f)
    surface_area = result["coverage"]
    surface_areas.append(surface_area)

df_surface_areas = pd.DataFrame({"sample_id": range(len(surface_areas)), "surface_area": surface_areas})
df_surface_areas.head()

In [ ]:
display_images(result_images_dict) 

In [ ]:
# merge dataframes on sample_id
df_merge = pd.merge(df_info, df_surface_areas, on="sample_id")
df_merge.head()

In [ ]:
# convert reference_surface_areas to dataframe and merge with df
df_reference_surface_areas = pd.DataFrame({"reference_id": list(reference_surface_areas.keys()), "reference_surface_area": list(reference_surface_areas.values())})
df_reference_surface_areas.head()

In [ ]:
# merge dataframes on reference_id
df = pd.merge(df_merge, df_reference_surface_areas, on="reference_id")
df.head()

In [ ]:
# create a column that is surface area divided by reference surface area
df["coverage"] = df["surface_area"] / df["reference_surface_area"]
df.head()

In [ ]:
def reference_id_to_cloth_type(reference_id):
    if reference_id <= 14:
        return "shirt"
    else:
        return "towel"
    
# add a column for cloth type
df["cloth_type"] = df["reference_id"].apply(reference_id_to_cloth_type)

In [ ]:
import matplotlib.pyplot as plt

# Filter the DataFrame for each cloth type
shirts = df[df["cloth_type"] == "shirt"]["coverage"]
towels = df[df["cloth_type"] == "towel"]["coverage"]

# Create the figure and axes
plt.figure(figsize=(12, 6))
ax = plt.gca()  # Get the current Axes for customization

# Plot the histograms
plt.hist(shirts, bins=20, alpha=0.5, label="Shirts", color="skyblue", edgecolor="white")
plt.hist(towels, bins=20, alpha=0.5, label="Towels", color="lightcoral", edgecolor="white")

# Add titles and labels
plt.title("Coverage Distribution by Cloth Type", fontsize=14)
plt.xlabel("Coverage", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.legend(fontsize=12)  # Add a legend to distinguish the histograms

# Customize appearance
plt.grid(axis='y', alpha=0.75)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Show the plot
plt.show()


In [ ]:
df["coverage"].describe()

In [ ]:
df["grasp_success"].describe()

In [ ]:
# average grasp success rate
df["grasp_success"].mean()

In [ ]:
df.groupby("cloth_type")["coverage"].mean()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Filter data for success and failure
success_coverages = df[df['grasp_success'] >= 0.5]['coverage']  # Assuming success if >= 0.5
failure_coverages = df[df['grasp_success'] < 0.5]['coverage']

# Create figure and axes
plt.figure(figsize=(12, 6))
ax = plt.gca()

# Plot histograms with different colors for clarity
plt.hist(success_coverages, bins=20, alpha=0.5, label="Success", color="green", edgecolor="white")
plt.hist(failure_coverages, bins=10, alpha=0.5, label="Failure", color="red", edgecolor="white")

# Add labels, title, and legend
plt.title("Coverage Distribution by Grasp Success", fontsize=14)
plt.xlabel("Coverage", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.legend(fontsize=12)

# Customize appearance
plt.grid(axis='y', alpha=0.75)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Show the plot
plt.show()


In [ ]:
import numpy as np

# Filter data for success and failure (same as before)
success_coverages = df[df['grasp_success'] >= 0.5]['coverage']
failure_coverages = df[df['grasp_success'] < 0.5]['coverage']

# Calculate mean and standard deviation
success_mean = np.mean(success_coverages)
success_std = np.std(success_coverages)
failure_mean = np.mean(failure_coverages)
failure_std = np.std(failure_coverages)

# Display the results (formatted for clarity)
print(f"Success - Mean Coverage: {success_mean:.2f}, Std Dev: {success_std:.2f}")
print(f"Failure - Mean Coverage: {failure_mean:.2f}, Std Dev: {failure_std:.2f}")

In [ ]:
import pandas as pd

# Group by reference_id and calculate average and standard deviation
average_coverage_per_item = df.groupby("reference_id")["coverage"].agg(
    ["mean", "std"]
)

# Rename columns for clarity
average_coverage_per_item.columns = ["Mean Coverage", "Std Dev Coverage"]

# Round values to 2 decimal places
average_coverage_per_item = average_coverage_per_item.round(2)

# Display in a formatted table (markdown)
print(average_coverage_per_item.to_markdown(numalign="left", stralign="left"))

In [ ]:
import pandas as pd

# Define a function to calculate the success rate.
def success_rate(series):
    return (series >= 0.5).mean()

# Group by 'reference_id' and calculate success rate.
success_rate_per_item = df.groupby('reference_id')['grasp_success'].agg(success_rate)

# Rename the column.
success_rate_per_item.name = 'Success Rate'

# Round the values to 3 decimal places and multiply by 100 to get a percentage.
success_rate_per_item = (success_rate_per_item * 100).round(0)

# Print the results in markdown.
print(success_rate_per_item.to_markdown(numalign='left', stralign='left'))

In [ ]:
# Create a new column to represent success or failure
df["grasp_result"] = df["grasp_success"].apply(
    lambda x: "Success" if x >= 0.5 else "Failure"
)  # Assuming success if >= 0.5, adjust if needed


# Group by cloth_type and grasp_result, and calculate average coverage
average_coverage_per_cloth_type = df.groupby(["cloth_type", "grasp_result"])[
    "coverage"
].mean()

# Convert Series to DataFrame for better formatting
average_coverage_per_cloth_type_df = average_coverage_per_cloth_type.reset_index()

# Pivot the DataFrame to get grasp_result as columns
average_coverage_per_cloth_type_df = average_coverage_per_cloth_type_df.pivot(
    index="cloth_type", columns="grasp_result", values="coverage"
)

average_coverage_per_cloth_type_df = average_coverage_per_cloth_type_df.round(2)


# Display the results in a table
print(average_coverage_per_cloth_type_df.to_markdown(numalign='left', stralign='left'))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt



# Create the swarm plot
plt.figure(figsize=(3.5, 8), dpi=100)  # Adjust the figure size as needed

sns.violinplot(y="coverage", data=df, density_norm="area", color="#FFFFFF", bw_adjust=0.8, linecolor="#c9c9c9", inner=None)


palette = ["#ed5651", "#4dbf6b"]

sns.swarmplot(y="coverage", data=df, hue='grasp_success', size=8, palette=palette, warn_thresh=0.0)
# sns.swarmplot(x="team_name", y="coverage", data=merged_df, order=teams, hue='cloth_type', size=6)

# Add title and labels
plt.title("Coverage Distribution", fontsize=14)
plt.xlabel("")
plt.ylabel("Coverage", fontsize=12)
plt.xticks(rotation=20)  # Rotate x-axis labels if needed

plt.legend(title='Successful grasp', bbox_to_anchor=(1, 1), loc='upper left') 

plt.grid(axis='y', alpha=0.25)  # 'alpha' controls the transparency of the grid lines


# make the y-axis 0 to 1
plt.ylim([0, 1.03])

In [ ]:
# find the sample with minimum coverage
min_coverage = df["coverage"].min()
min_coverage_sample = df[df["coverage"] == min_coverage]
min_coverage_sample